The cell below is importing libraries that I expect to use throughout this lab. 

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

The next two cells are importing Beautiful Soup 4 and scraping the Postal Codes from the Wikipedia page.

In [3]:
import bs4

In [4]:
from bs4 import BeautifulSoup
import requests
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
#article = soup.find('table',class_='wikitable sortable')
table = soup.find('table',class_='wikitable sortable')
table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
df = pd.DataFrame(res, columns=["PostalCode", "Borough", "Neighborhood"])
export_csv = df.to_csv (r'export_dataframe.csv', encoding='utf-8', index = None, header=True)
print(df)

    PostalCode           Borough  \
0          M1A      Not assigned   
1          M2A      Not assigned   
2          M3A        North York   
3          M4A        North York   
4          M5A  Downtown Toronto   
5          M5A  Downtown Toronto   
6          M6A        North York   
7          M6A        North York   
8          M7A      Queen's Park   
9          M8A      Not assigned   
10         M9A         Etobicoke   
11         M1B       Scarborough   
12         M1B       Scarborough   
13         M2B      Not assigned   
14         M3B        North York   
15         M4B         East York   
16         M4B         East York   
17         M5B  Downtown Toronto   
18         M5B  Downtown Toronto   
19         M6B        North York   
20         M7B      Not assigned   
21         M8B      Not assigned   
22         M9B         Etobicoke   
23         M9B         Etobicoke   
24         M9B         Etobicoke   
25         M9B         Etobicoke   
26         M9B         Etobi

The cell below is for creating the data frame from the csv that was created during the scrape. 

In [6]:
df2=pd.read_csv('export_dataframe.csv') 
print(df2)

    PostalCode           Borough  \
0          M1A      Not assigned   
1          M2A      Not assigned   
2          M3A        North York   
3          M4A        North York   
4          M5A  Downtown Toronto   
5          M5A  Downtown Toronto   
6          M6A        North York   
7          M6A        North York   
8          M7A      Queen's Park   
9          M8A      Not assigned   
10         M9A         Etobicoke   
11         M1B       Scarborough   
12         M1B       Scarborough   
13         M2B      Not assigned   
14         M3B        North York   
15         M4B         East York   
16         M4B         East York   
17         M5B  Downtown Toronto   
18         M5B  Downtown Toronto   
19         M6B        North York   
20         M7B      Not assigned   
21         M8B      Not assigned   
22         M9B         Etobicoke   
23         M9B         Etobicoke   
24         M9B         Etobicoke   
25         M9B         Etobicoke   
26         M9B         Etobi

The cell below is grouping the Neighborhoods into their appropriate Postal Code and Borough.

In [8]:
df_grouped = df2.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
print(df_grouped)

    PostalCode           Borough  \
0          M1A      Not assigned   
1          M1B       Scarborough   
2          M1C       Scarborough   
3          M1E       Scarborough   
4          M1G       Scarborough   
5          M1H       Scarborough   
6          M1J       Scarborough   
7          M1K       Scarborough   
8          M1L       Scarborough   
9          M1M       Scarborough   
10         M1N       Scarborough   
11         M1P       Scarborough   
12         M1R       Scarborough   
13         M1S       Scarborough   
14         M1T       Scarborough   
15         M1V       Scarborough   
16         M1W       Scarborough   
17         M1X       Scarborough   
18         M1Y      Not assigned   
19         M1Z      Not assigned   
20         M2A      Not assigned   
21         M2B      Not assigned   
22         M2C      Not assigned   
23         M2E      Not assigned   
24         M2G      Not assigned   
25         M2H        North York   
26         M2J        North 

The cell below is eliminating the Boroughs with a value of Not Assigned

In [9]:
df3 = df_grouped[df_grouped.Borough != 'Not assigned']
print(df3)

    PostalCode           Borough  \
1          M1B       Scarborough   
2          M1C       Scarborough   
3          M1E       Scarborough   
4          M1G       Scarborough   
5          M1H       Scarborough   
6          M1J       Scarborough   
7          M1K       Scarborough   
8          M1L       Scarborough   
9          M1M       Scarborough   
10         M1N       Scarborough   
11         M1P       Scarborough   
12         M1R       Scarborough   
13         M1S       Scarborough   
14         M1T       Scarborough   
15         M1V       Scarborough   
16         M1W       Scarborough   
17         M1X       Scarborough   
25         M2H        North York   
26         M2J        North York   
27         M2K        North York   
28         M2L        North York   
29         M2M        North York   
30         M2N        North York   
31         M2P        North York   
32         M2R        North York   
40         M3A        North York   
41         M3B        North 

The cell below is showing the dimensions of the resulting dataframe. 

In [10]:
df3.shape

(103, 3)